# What I've done

## Preprocessing

I cleared the errors and used linear interpolation.

## Yearly data

I also have it so each year is saved as its own variable, just use the y + the year you want. for example y1990.

There is also a dictionary called yearly_data containing yearly data. You can also call a single years data like this: yearly_data[1995] 

## Monthly data
Monthly Data is stored in the all_data dataframe.

# Complete Code

# Code Broken up and explained

## Data Scrub

In [1]:
import pandas as pd

### Arctic Ice

In [2]:
# lists csv file then combines them into a pandas dataframe
csv_files = ['N_01_extent_v3.0.csv', 'N_02_extent_v3.0.csv', 'N_03_extent_v3.0.csv',
              'N_04_extent_v3.0.csv', 'N_05_extent_v3.0.csv', 'N_06_extent_v3.0.csv',
              'N_07_extent_v3.0.csv', 'N_08_extent_v3.0.csv', 'N_09_extent_v3.0.csv',
              'N_10_extent_v3.0.csv', 'N_11_extent_v3.0.csv', 'N_12_extent_v3.0.csv']

raw_ds = pd.concat([pd.read_csv(file) for file in csv_files ], ignore_index=True)

In [3]:
#I change some column headings to remove special characters
import pandas as pd

raw_ds.columns = raw_ds.columns.str.replace('-', '')
raw_ds.columns = raw_ds.columns.str.replace(' ', '')

In [4]:
# The dataframe is sorted by year, then month. Reorganizing the data month by month. 
raw_ds = raw_ds.sort_values(by=['year', 'mo'])
raw_ds = raw_ds.reset_index(drop=True)

In [5]:
#seperates into analysis_df, which is to be used for analysis, and 2023_df.
#2023 data was seperated into as it uses a new datatype this could
#prove an issue as all other data is Godddard type

a = raw_ds.loc[62:529]
df_2023 = raw_ds.loc[530:]

### Temp

In [6]:
#takes temp data from csv file and drops irrelvant columns
t = pd.read_csv('Temps.csv')
t = t[['TAVG']]
t = t.rename(columns={"TAVG": "temp"})

#cuts down the data to the years and months covered by arctiv ice data
t =  t.loc[822:1289]

### Methane

In [7]:
#takes temp data from csv file and drops irrelvant columns
m = pd.read_csv('Methane.csv')
m = m[['average']]
m = m.rename(columns={"average": "methane"})

#cuts down the data to the years and months covered by arctiv ice data
m =  m.loc[6:473] 

In [8]:
#takes temp data from csv file and drops irrelvant columns
c = pd.read_csv('Carbon_Dioxide.csv')
c = c[['average']]
c = c.rename(columns={"average": "CO2"})

#cuts down the data to the years and months covered by arctiv ice data
#m =  m.loc[6:473] 
c =  c.loc[60:527]

## Merging

In [9]:
a, m, t, c = a.reset_index(), m.reset_index(), t.reset_index(), c.reset_index()

all_data = pd.concat([a, c, m, t,], axis=1, join="inner")
all_data = all_data.drop(['index','datatype', 'region'], axis=1)

all_data

year  mo  extent   area     CO2  methane   temp
0    1984   1   14.47  11.68  344.32  1638.77 -26.27
1    1984   2   15.30  12.26  344.82  1638.87 -36.08
2    1984   3   15.58  12.55  344.96  1640.82 -26.93
3    1984   4   15.01  12.18  345.19  1643.96 -23.68
4    1984   5   13.58  10.88  345.33  1643.16  -8.59
..    ...  ..     ...    ...     ...      ...    ...
463  2022   8    5.95   4.03  414.41  1908.78   4.09
464  2022   9    4.90   3.47  414.63  1915.46   1.91
465  2022  10    6.66   5.25  416.14  1920.11  -4.07
466  2022  11    9.73   8.52  417.77  1923.75  -9.97
467  2022  12   11.89  10.27  418.80  1924.54 -15.07

[468 rows x 7 columns]

In [10]:
all_data = all_data.rename(columns={"mo": "month"})
all_data

year  month  extent   area     CO2  methane   temp
0    1984      1   14.47  11.68  344.32  1638.77 -26.27
1    1984      2   15.30  12.26  344.82  1638.87 -36.08
2    1984      3   15.58  12.55  344.96  1640.82 -26.93
3    1984      4   15.01  12.18  345.19  1643.96 -23.68
4    1984      5   13.58  10.88  345.33  1643.16  -8.59
..    ...    ...     ...    ...     ...      ...    ...
463  2022      8    5.95   4.03  414.41  1908.78   4.09
464  2022      9    4.90   3.47  414.63  1915.46   1.91
465  2022     10    6.66   5.25  416.14  1920.11  -4.07
466  2022     11    9.73   8.52  417.77  1923.75  -9.97
467  2022     12   11.89  10.27  418.80  1924.54 -15.07

[468 rows x 7 columns]

## Error removal

replaces errors with nans

In [11]:
all_data.loc[43, 'area'] = pd.NA
all_data.loc[47, 'area'] = pd.NA
all_data.loc[47, 'extent'] = pd.NA
all_data.loc[48, 'area'] = pd.NA
all_data.loc[48, 'extent'] = pd.NA

#all_data.describe()

Uses linear interpolation to replace the nans

In [12]:
all_data.interpolate(method='linear', axis=0, limit=None, inplace=True)

# Yearly data

### THE CELL BELOW REMOVES THE FEATURES NOT INCLUDED IN THE ARCTIC ICE DATATSET

In [13]:
all_data = all_data.drop(["CO2", "methane", "temp"], axis=1)

### THE CELL ABOVE REMOVES THE FEATURES NOT INCLUDED IN THE ARCTIC ICE DATATSET

In [14]:


y_names = list(range(1984,2023))
y_data = []

for i in range(len(y_names)):
    appended = all_data.loc[lambda df: df['year'] == y_names[i]]
    y_data.append(appended)

yearly_data = dict(zip(y_names, y_data))

You can call a single yeards data like this: yearly_data[1995]

In [15]:
yearly_data[1995]

year  month  extent   area
132  1995      1   14.59  12.79
133  1995      2   15.23  13.37
134  1995      3   15.26  13.34
135  1995      4   14.45  12.40
136  1995      5   12.97  10.90
137  1995      6   11.44   8.94
138  1995      7    8.99   6.16
139  1995      8    6.74   4.69
140  1995      9    6.08   4.43
141  1995     10    7.83   5.99
142  1995     11   10.76   9.03
143  1995     12   12.92  11.09

I also have it so each year is saved as its own variable, just use the y + the year you want. Example below.

In [16]:
for i in range(len(y_names)):
    
    Dynamic_Variable_Name = ('y' + str(y_names[i]))
    
    locals()[Dynamic_Variable_Name] = yearly_data[y_names[i]].copy(deep=True)

In [17]:
y1990

year  month  extent   area
72  1990      1   14.78  12.80
73  1990      2   15.58  13.42
74  1990      3   15.87  13.51
75  1990      4   14.65  12.28
76  1990      5   13.23  10.90
77  1990      6   11.64   9.20
78  1990      7    9.25   6.52
79  1990      8    6.80   4.99
80  1990      9    6.14   4.55
81  1990     10    8.48   6.74
82  1990     11   11.08   9.64
83  1990     12   13.11  11.47